참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 05.004 보에서의 전단응력

### a. 해석

* 보의 $x$ 축에 수직인 단면 $A$ 를 생각해 보자.
* y 축에 수직인 평면으로 $A$ 를 둘로 나눌 수 있다.
* 그 중 위쪽을 $A'$라고 하자.
* $A'$에 $x$ 방향으로 가해지는 힘을 생각해 보자.
$$
P = \int_{A'} dP = \int_{A'} \sigma dA
$$
* 굽힘 응력 $\sigma$ 는 중립면으로부터의 거리 $y$에 따라 변한다. (p.170 eq. 5.3)
$$
\sigma = - \frac{M}{I} y
$$
* 힘 $P$를 구하기 위한 적분에 반영하면 다음과 같다. (p.197 eq. 5.6)
$$
P = \int_{A'} - \frac{M}{I} y dA = - \frac{M}{I} \int_{A'} y dA
$$
* 여기서 다음과 같이 $A'$의 중립축에 대한 1차 모멘트 $Q$를 정하자. (p.197 eq. 5.7a)
$$
Q = \int_{A'} y dA
$$
* $Q$를 이용하면 $P$는 다음과 같이 표시된다.
$$
P = - \frac{MQ}{I}
$$

* $y$를 면적 전체에 대해 적분하면 0이 된다. (p. 169 b)
$$
0 = \int_{A} y dA
$$

#### $Q$ 값의 변화

* $b \times h$ 직사각형 단면의 경우 $A'$ 의 아랫변과 중립면까지의 거리 $y'$ 에 따른 Q 값의 변화를 알아 보자.

In [ ]:
b_m, h_m = sy.symbols('b[m], h[m]', positive=True)
s_d = {b_m: 120e-3,
      h_m: 200e-3}

* 미소 면적을 다음과 같이 표시할 수 있다.
$$
dA = b \cdot dy
$$
* $Q$에 대한 적분에 반영하자.
$$
\begin{align}
Q &= \int_{A'} y dA = \int_{y=y'}^{y=\frac{h}{2}} y b \cdot dy \\
&= b \int_{y=y'}^{y=\frac{h}{2}} y \cdot dy
\end{align}
$$

In [ ]:
y_m, y_prime_m = sy.symbols('y[m], y_{prime}[m]', real=True)
Q = b_m * sy.integrate(y_m, (y_m, y_prime_m, h_m / 2))
Q

예제 5.1 의 단면의 예를 들면 다음과 같이 될 것이다.

In [ ]:
Q.subs(s_d)

선도로 표시하면 다음과 같이 될 것이다.

In [ ]:
splot.plot_parametric((Q.subs(s_d), y_prime_m), (y_prime_m, -h_m.subs(s_d)/2, h_m.subs(s_d)/2),
          ylabel='$y\'(m)$', xlabel='$Q(m^3)$')

### b. 수평 전단 응력

사용할 변수들을 미리 준비해 둔다.

In [ ]:
I, M, Q, V = sy.symbols('I, M, Q, V', real=True)

In [ ]:
dM, dF, dx = sy.symbols('dM, dF, dx', real=True)

In [ ]:
tau = sy.symbols('tau', real=True)

* 보 내부의 수평면에 작용하는 전단응력을 생각해 보자.

* 길이 $dx$ 인 미소 보 요소를 중립면으로부터 $y'$ 위에 있는 수평면으로 자르는 것을 생각해 보자.
* 굽힘모멘트가 작용할 때, 윗쪽 부분에 $x$ 방향으로 걸리는 힘이 위에서 구한 $P$가 될 것이다.
$$
P = - \frac{Q}{I}M
$$


In [ ]:
P_left = -Q * M / I
P_left

* 미소 보 요소의 $x$ 지점에 걸리는 굽힘 모멘트가 $M$이라면, $x+dx$ 지점에 작용하는 모멘트는 $M+dM$이 될 것이다. 따라서 $x+dx$ 지점에 작용하는 힘 $P+dP$ 는 다음과 같을 것이다.
$$
P+dP = - \frac{Q}{I}(M+dM)\\
$$

In [ ]:
P_right = -(Q / I) * (M + dM)
P_right

* 힘 평형 방정식

In [ ]:
F_eq = sy.Eq(P_right - P_left+dF, 0)
F_eq

* 위 방정식으로 부터 dF 를 구한다.

In [ ]:
dF_sol_list = sy.solve(F_eq, dF)
dF_sol_list

* 전단응력 방정식

In [ ]:
tau_eq = sy.Eq(tau * b_m * dx,  dF_sol_list[0])
tau_eq

* 전단응력을 구하면 다음과 같다.

In [ ]:
tau_sol_list = sy.solve(tau_eq, tau)
tau_sol_list

* $V=\frac{dM}{dx}$ (p. 145 eq. 4.2) 임을 이용하면 eq. 5.8 (p. 199) 을 얻을 수 있다.

In [ ]:
tau_sol_list[0].xreplace({dM: V * dx})